In [1]:
import torch

In [2]:
import pandas
train_data = pandas.read_csv('./data/train.csv')
test_data = pandas.read_csv('./data/test.csv')

In [3]:
val_data = train_data[:int(len(train_data)/10)]
train_data = train_data[int(len(train_data)/10):]

In [17]:
import numpy

train_y = torch.nn.functional.one_hot(torch.Tensor(train_data["label"].values).to(torch.int64), 10)
train_x = train_data.loc[:, train_data.columns != "label"]
train_x /= 256
train_x = torch.reshape(torch.Tensor(train_x.values), (37800, 28, 28))

val_y = torch.nn.functional.one_hot(torch.Tensor(val_data["label"].values).to(torch.int64), 10)
val_x = val_data.loc[:, val_data.columns != "label"]
val_x /= 256
val_x = torch.reshape(torch.Tensor(val_x.values), (4200, 28, 28))

In [26]:
train = train_x[0].unsqueeze(0)
print(torch.nn.Conv1d(in_channels=28, out_channels=26, kernel_size=3)(train).shape)

torch.Size([1, 26, 26])


In [42]:
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(in_channels=28, out_channels=26, kernel_size=3)
        self.pool = torch.nn.MaxPool2d(2)
        self.flatten = torch.nn.Flatten(0, -1)
        self.linear = torch.nn.Linear(169, 64)
        self.drop = torch.nn.Dropout(0.2)
        self.linear1 = torch.nn.Linear(64,10)
        self.fina = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.drop(x)
        x = self.linear1(x)
        x = self.fina(x)
        return x

net = Net()

In [43]:
print(net)

Net(
  (conv1): Conv1d(28, 26, kernel_size=(3,), stride=(1,))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=0, end_dim=-1)
  (linear): Linear(in_features=169, out_features=64, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=64, out_features=10, bias=True)
  (fina): Sigmoid()
)


In [44]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
params = net.parameters()
optimizer = optim.Adam(params)

In [46]:
for epochs in range(2):
    running_loss = 0.0
    
    for i, data in enumerate(train_x):
        inpu = train_x[i]
        labels = train_y[i].float()
#         print(inpu)
        
        optimizer.zero_grad()
        
        inp = inpu.unsqueeze(0)
        outputs = net(inp)
#         console.log(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss  += loss.item()
        if(i%2000 == 1999):
            print(running_loss/2000)
            running_loss = 0.0
print("Finished training")

0.013230233238317197
0.01374399767235517
0.012312154093872237
0.01405778315169942
0.014041347010951765
0.011607471524468201
0.012679349678092757
0.012922946990869873
0.01325137742246179
0.012612659603480004
0.014133176704992898
0.014378128454440628
0.013314133286660298
0.012706881280625713
0.01409408628188938
0.013821486082770009
0.012638738779556298
0.012815883719387099
0.014154343382282086
0.015073057994258116
0.013642943949975977
0.013647981114754168
0.014395348513157257
0.012031420143544743
0.012864453273956281
0.013011323018137191
0.014912820346253856
0.012886522158487854
0.013675815504902397
0.013931940451939616
0.01312094574081494
0.012594917915530364
0.012578835676365347
0.013364599491948573
0.014063573324005754
0.011936488357136975
Finished training
